In [24]:
from __future__ import division
import os
import urllib, cStringIO
import pymongo as pm ## first establish ssh tunnel to server where database is running
import base64
import numpy as np
from numpy import *
import PIL
from PIL import Image
import base64
import matplotlib
%matplotlib inline
from matplotlib import pylab, mlab, pyplot
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import seaborn as sns
sns.set_context('poster')
sns.set_style('white')
from matplotlib.path import Path
import matplotlib.patches as patches
import cv2
import pandas as pd
from svgpathtools import parse_path
import svg_distance_helpers as rsh
import svg_render_helpers as rrh

#### directory and file hierarchy

In [25]:
# directory & file hierarchy
analysis_dir = os.getcwd()
tracing_ref_pixel = os.path.join(analysis_dir, 'tracing_ref_pixel')
tracing_ref_svg = os.path.join(analysis_dir, 'tracing_ref_svg')
data_dir = os.path.join(analysis_dir,"trace_test")

canvas_side = 432 # image size
rows, cols = canvas_side, canvas_side # the size of a given drawing image

#### Tracing/Copying Error Calculation 

In [28]:
# # change names. only for testing purpose
# for index, imgfile in enumerate(os.listdir(data_dir)):
#     new_filename = 'test{:02}.png'.format(index)
#     os.rename(os.path.join(data_dir,imgfile), os.path.join(data_dir,new_filename))

In [29]:
# os.listdir(data_dir)

In [6]:
# reload(rsh)
# # get tracings from the data folder
# #for file in os.listdir(data_dir):

# # read in tracing and reference files
# fpath = os.path.join(data_dir, 'test01.png')
# img_draw = cv2.imread(fpath)
# img_draw = rsh.white_color_to_num(img_draw)

# square_ref_file = os.path.join(tracing_ref_pixel, 'resize_square.png')
# img_ref = cv2.imread(square_ref_file)
# img_ref = rsh.black_color_to_num(img_ref)

#### set up a toy case where the ref and trace are identical and simple

In [32]:
import torch
from torch.autograd import Variable
import torch.nn as nn

class LinearTransform(torch.nn.Module):
    def __init__(self, size):
        super(LinearTransform, self).__init__()
        self.transform = torch.nn.Linear(size, size, bias=True)  # two in and two out
        # init the model with the identity transformation matrix
        self.transform.weight = torch.nn.Parameter(torch.eye(size))
        self.transform.bias = torch.nn.Parameter(torch.zeros(size))

    def forward(self, x):
        y_pred = self.transform(x)
        return y_pred

In [33]:
## 5x5 image where there is a horizontal bar in the second row
img_ref = np.zeros((5,5))
img_ref[1,1:5] = 1

img_draw = np.zeros((5,5))
img_draw[2,1:5] = 1

In [34]:
## get x_data and y_data in the appropriate format
num_rows, num_cols = img_ref.shape[0], img_ref.shape[1]
end_index_1d = num_rows * num_cols - 1 # the unique id of the last pixel in both ref and draw

draw_pixels = rsh.find_black_pixels(img_draw) # a list of pixels that are black  2 x k
x_data = Variable(torch.tensor(draw_pixels, dtype=torch.float, ))
y_data = Variable(torch.tensor(img_ref, dtype=torch.float).view(-1)) # 1 x end_index_1d.

In [21]:
# init model
model = LinearTransform(2)  # weight 2 x 2   bias 1 x 2
# set up optimizer
lr = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
# number of training steps
num_train_steps = 5

## print model params
# list(model.parameters())[0].data.numpy()

loss = torch.tensor(0.0, dtype=torch.float, requires_grad=True)

# thresholding
threshold = nn.Hardtanh(0, end_index_1d) # change everything smaller than 0 to 0 and larger than end_index to end_index

In [23]:
for j,epoch in enumerate(range(num_train_steps)):
    x_prime = model(x_data) # 2 x k
    x_prime_1d = rsh.pixel_list_to_1d(x_prime, num_cols) # 1 x 2k
#     print '{}'.format(x_prime)
#     print '{}'.format(x_prime_1d)    
    
    # threshold image
    cropped = threshold(x_prime_1d)
#     print 'cropped {} {}'.format(cropped, cropped.requires_grad)
        
    # get indices of on pixels in thresholded image
    cropped_index = threshold(x_prime_1d).type(torch.long).requires_grad_()
#     print 'cropped_index {} {}'.format(cropped_index, cropped_index.requires_grad)
    
    # index selection
    ref_select = torch.index_select(y_data, 0, cropped_index).requires_grad_() # get color values on img_ref with overlapping indices
    print 'ref_select {} {}'.format(ref_select, ref_select.requires_grad)
    loss = -1e3 * torch.mean(ref_select)  
    
    # Zero gradients, perform a backward pass,
    # and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
#     print 'model params: weight = {}; bias = {}'.format(model.transform.weight, model.transform.bias)
    print 'loss = {} grad = {}'.format(loss, loss.grad)

ref_select tensor([ 0.,  0.,  0.,  0.]) True
loss = -0.0 grad = None
ref_select tensor([ 0.,  0.,  0.,  0.]) True
loss = -0.0 grad = None
ref_select tensor([ 0.,  0.,  0.,  0.]) True
loss = -0.0 grad = None
ref_select tensor([ 0.,  0.,  0.,  0.]) True
loss = -0.0 grad = None
ref_select tensor([ 0.,  0.,  0.,  0.]) True
loss = -0.0 grad = None


In [ ]:
reload(rsh)
loss, final_draw, weights, bias = rsh.minimize_shape_error(img_ref, img_draw)
print loss
print weights
print bias

#### preprocessing of tracing and visualization (if desired)


In [ ]:
reallyRun = 1
if reallyRun:
    rsh.plot_shape(verts_list, codes_list, canvas_side)
    #rsh.plot_stroke(verts_list[1], codes_list[1], canvas_side)

#### Prepare human rating

In [ ]:
data = pd.DataFrame(os.listdir(data_dir))
data.columns = ['image_id']
data.to_csv(os.path.join(analysis_dir,'performance.csv'))